# Pattern Databases

## Imports

These are the imports needed to solve this task:

In [1]:
from heapq import heappop, heappush
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
from tests import case1, case2, case3, case4
import numpy as np
import matplotlib.pyplot as plt
from random import choice

%matplotlib notebook

## Test cases

These are the test cases that will be used to run and compare the algorithms:

In [2]:
cube1 = Cube(case1)
cube2 = Cube(case2)
cube3 = Cube(case3)
cube4 = Cube(case4)

To generate all the nodes, we will take a solved "dummy cube" and start the exploration from there

In [3]:
dummy_cube = Cube(scrambled=False)
discovered = {}

In [4]:
# generate all neighbours possible using all possible moves

def get_neighbours(cube):
    return [Cube.move_state(cube.clone_state(), move) for move in range(6)]

In [5]:
# we add all the possible neighbours into a queue and visit them one by one;
# the depth (distance from the goal state to that state) will be the depth of
# the parent plus 1
def generate_catalog(node):
    queue = []
    queue.append((node.clone(), 0))
    while queue:
        current_node, current_cost = queue.pop(0)
        
        # if we reached a depth that is bigger than 7 or the node is already
        # into the queue, we skip the iteration
        if current_cost > 7 or current_node.hash() in discovered:
            continue
        
        discovered[current_node.hash()] = current_cost
        
        # now we add all of the neighbours of the current node into the queue
        # so we can visit them
        for neighbours in get_neighbours(current_node):
            probe_cube = Cube(scrambled=False)
            probe_cube.state = neighbours
            queue.append((probe_cube, current_cost + 1))

In [6]:
%%time
generate_catalog(Cube(scrambled=False))
print(len(discovered))
# http://anttila.ca/michael/devilsalgorithm/ -> this is the site that helped me get the correct number of total states

44971
CPU times: total: 10.1 s
Wall time: 10.1 s


In [7]:
# heuristic function used for A*
def heuristic_score_updated(cube):
    if cube.hash() in discovered:
        return discovered[cube.hash()]
    
    score = 0
    for i in range(len(cube.state)):
        if cube.goal_state[i] != cube.state[i]:
            score += 1
    
    return score / 8

In [8]:
# heuristic function used for MCST
# to make it a reward, if the node is found in the dictionary, it will return
# the result of the maximum possible distance from the solution (7) minus the
# actual distance; in this way, a state that is 7 moves away from the goal
# state will have a reward of 0
def heuristic_score_updated_mcts(cube):
    if cube.hash() in discovered:
        return  7 - discovered[cube.hash()]
    
    score = 0
    for i in range(len(cube.state)):
        if cube.goal_state[i] != cube.state[i]:
            score += 1
    
    return 3 - score / 8

In [9]:
def is_solved(cube):
    comparator = cube.goal_state == cube.state
    return comparator.all()

In [10]:
# the A* algorithn
def astar(cube):
    priority_queue = []
    heappush(priority_queue, (0 + heuristic_score_updated(cube), cube.clone()))
    discovered = {cube.hash(): (None, 0)}
    
    while priority_queue:
        current_cost, current_node = heappop(priority_queue)
        current_node_hash =current_node.hash() 
        
        if is_solved(current_node):
            print("solved cube!")
            break
        else:
            for neighbour in get_neighbours(current_node):
                # we iterate trough all the possible neighbours from this
                # current state
                probe_cube = Cube(scrambled=False)
                probe_cube.state = neighbour
                probe_cube_hash = probe_cube.hash()
                
                # calculate the cost
                new_cost = discovered[current_node_hash][1] + 1
                
                # if the state is not inside the hash map or its new cost is smaller
                # compared to what we have in the hash map, we add/update it
                if probe_cube_hash not in discovered or new_cost < discovered[probe_cube_hash][1]:
                    discovered[probe_cube_hash] = (current_node_hash, new_cost)
                    heappush(priority_queue, (new_cost + heuristic_score_updated(probe_cube), probe_cube))
    
    
    
    solution = []
    current_node = current_node.hash()
    while current_node is not None:
        solution.append(current_node)
        current_node, _ = discovered[current_node]
    path_len = len(solution) - 1
    discovered_nodes = len(discovered)
    
    print(f"Total number of nodes discovered: {discovered_nodes}")
    print(f"Length of the solution is: {path_len}")
    print(f"Solution is:\n")
    solution = solution[::-1]
    for sol in solution:
        print(f"\t{sol}")
    
    return discovered_nodes, path_len

In [11]:
%%time
discovered_nodes_case1_astar, path_len_case1_astar = astar(cube1)

solved cube!
Total number of nodes discovered: 27
Length of the solution is: 5
Solution is:

	255011422234350044103135
	502511322234300144145035
	500111252232303444143550
	550021514242303440113352
	050511114242333340405252
	000011112222333344445555
CPU times: total: 0 ns
Wall time: 1.99 ms


In [12]:
%%time
discovered_nodes_case1_astar, path_len_case1_astar = astar(cube2)

solved cube!
Total number of nodes discovered: 2400
Length of the solution is: 7
Solution is:

	115544133204352340002215
	105043415224352340032115
	001513215224332040445315
	002113245220331540443551
	052121434200331540413255
	051521214243330040415352
	001111222233330044445555
	000011112222333344445555
CPU times: total: 156 ms
Wall time: 153 ms


In [13]:
%%time
discovered_nodes_case1_astar, path_len_case1_astar = astar(cube3)

solved cube!
Total number of nodes discovered: 69962
Length of the solution is: 9
Solution is:

	311401145205353242254003
	321514013205353240254104
	253114413205353240105204
	223441140205353245115003
	324211140205353045251403
	354514113245353040201202
	304041112225353044231505
	303041402211352544230155
	003311002211332244445555
	000011112222333344445555
CPU times: total: 4.88 s
Wall time: 4.87 s


In [24]:
%%time
discovered_nodes_case1_astar, path_len_case1_astar = astar(cube4)

solved cube!
Total number of nodes discovered: 786294
Length of the solution is: 11
Solution is:

	521252101210333445030445
	541515023250333442020141
	540215503234331542021104
	510451052224331544021303
	511551042205332444020133
	512451152204330544023031
	145201352204320044513531
	140001522235320444513315
	401031322235310544052415
	004141222235350044331515
	000041412222353544331155
	000011112222333344445555
CPU times: total: 1min
Wall time: 1min


In [15]:
# Constants

N = 'N'
Q = 'Q'
P = 'parent'
A = 'actions'

In [16]:
def init_node(parent=None):
    return {N: 0, Q: 0, P: parent, A: {}}

In [17]:
from math import sqrt, log
import numpy as np

def select_action(node, c):
    max_score = -1
    action = None
    
    for a, n in node[A].items():
        score = n[Q] / n[N] + c * sqrt(2 * log(node[N]) / n[N])
        if max_score < score:
            max_score = score
            action = a
    
    return action

In [18]:
def mcts(cube, budget, tree, heuristic, C):
    c = C
    count = 0
    solved = False
    if not tree:
        tree = init_node()
    
    for b in range(budget):
        
        probe_cube = cube.clone()
        node = tree
        solution = [] 
        while(not is_solved(probe_cube) and 
            all(move in node[A] for move in [x for x in range(6)])):
            selected_move = select_action(node, c)
            probe_cube.state = Cube.move_state(probe_cube.state, selected_move)
            solution.append(probe_cube.hash())
            node = node[A][selected_move]
        
        if is_solved(probe_cube):
            print("solved cube!")
            print(f"Total number of nodes discovered: {count}")
            print(f"Length of the solution is: {str(len(solution))}")
            print("Solution is:")
            print(solution)
            solved = True
            break
        else:
            new_move = choice(list(filter(lambda a: a not in node[A], [x for x in range(6)])))
                       
            probe_cube.state = Cube.move_state(probe_cube.state, new_move)
            node = init_node(node)
            count += 1
            node[P][A][new_move] = node
            
        tries = 0
        max_reward = 0
        while tries < 14:
            if max_reward < heuristic(probe_cube):
                max_reward = heuristic(probe_cube)
            
            if is_solved(probe_cube):
                break
                
            probe_cube.state = Cube.move_state(probe_cube.state, choice([x for x in range(6)]))
            tries += 1
            
        crt_node = node
        while crt_node:
            crt_node[N] += 1
            crt_node[Q] += max_reward
            crt_node = crt_node[P]
    
    
    if not solved:
        print('NO SOLUTION FOUND')

In [19]:
def run_case1(budget, H, C):
    mcts(cube1, budget, None, H, C)

def run_case2(budget, H, C):
    mcts(cube2, budget, None, H, C)
    
def run_case3(budget, H, C):
    mcts(cube3, budget, None, H, C)
    
def run_case4(budget, H, C):
    mcts(cube4, budget, None, H, C)

In [20]:
%%time
for i in range(20):
    run_case1(20000, heuristic_score_updated_mcts, 0.5)

solved cube!
Total number of nodes discovered: 38
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 32
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 37
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 36
Length of the solution is: 5
Solution is:
['502511322234300144145035', '500111252232303444143550', '550021514242303440113352', '050511114242333340405252', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 35
Length of the solu

In [21]:
%%time
for i in range(20):
    run_case2(20000, heuristic_score_updated_mcts, 0.5)

solved cube!
Total number of nodes discovered: 105
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 111
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215224332040445315', '002113245220331540443551', '052121434200331540413255', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 75
Length of the solution is: 7
Solution is:
['515104033204322240351415', '512204513203320440351154', '125214113203350340052454', '105541114243350340032252', '051521214243330040415352', '001111222233330044445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 80
Length of the solution is: 7
Solution is:
['105043415224352340032115', '001513215

In [22]:
%%time
for i in range(20):
    run_case3(20000, heuristic_score_updated_mcts, 0.5)

solved cube!
Total number of nodes discovered: 550
Length of the solution is: 9
Solution is:
['321514013205353240254104', '323214153201350540250441', '303545111241350540230242', '303351412221350544210045', '333021112221300044554545', '330021302211302144554455', '303041402211352544230155', '003311002211332244445555', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 454
Length of the solution is: 9
Solution is:
['321514013205353240254104', '323214153201350540250441', '343111545201350542220043', '413301045201320242155543', '410201335204320142154535', '410101025233320442153455', '041111525233330442420055', '040411115252333342425050', '000011112222333344445555']
solved cube!
Total number of nodes discovered: 613
Length of the solution is: 9
Solution is:
['321514013205353240254104', '323214153201350540250441', '343111545201350542220043', '303351412221350544210045', '333021112221300044554545', '330021302211302144554455', '303041402211352544230155', '00331100221133224

In [23]:
%%time
for i in range(20):
    run_case4(20000, heuristic_score_updated_mcts, 0.5)

NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
NO SOLUTION FOUND
CPU times: total: 5min 49s
Wall time: 5min 49s
